# Introduction 

Dans ce fichier nous allons mettre en place une implémentation basique de l'algorithme proposé dans l'article à étudier.

Nous allons pour l'instant étudier le dataset Extended Yale B.

Les données sont présentes dans le dossier suivant : data/yale/

# Imports 

In [49]:
import numpy as np
from PIL import Image
import os, sys

In [50]:
import os,sys

def read_images(path, sz=None): 
    c=0
    X,y = [], []
    for dirname , dirnames , filenames in os.walk(path):
        for subdirname in dirnames:
            subject_path = os.path.join(dirname , subdirname) 
            for filename in os.listdir(subject_path):
                try:
                    im = Image.open(os.path.join(subject_path , filename)) 
                    im = im.convert("L")
                    # resize to given size (if given)
                    if (sz is not None):
                        im = im.resize(sz, Image.ANTIALIAS)     
                    X.append(np.asarray(im, dtype=np.uint8)) 
                    y.append(c)
                except IOError:
                    print "I/O error({0}): {1}".format("errno", "strerror")
                except:
                    print "Unexpected error:", sys.exc_info()[0] 
                    raise
            c = c+1 
    return [X,y]

# Chargement des images 

In [51]:
X, y = read_images('data/yale/')

I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror
I/O error(errno): strerror


Vérification qu'on a bien chargé les données des individus

In [52]:
len(X)

1170

Alright : on a bien chargé 585 images qui ont été fournies, par individu (note : les erreurs correspondent aux fichiers .info).

*Rappel* : 9 faces + 64 (65?) illuminations par personne.